<a href="https://colab.research.google.com/github/hardik1408/CS771/blob/hardik/d3_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: open train_text_seq.csv

import pandas as pd
df_train = pd.read_csv('/content/train_text_seq.csv')
df_valid = pd.read_csv('/content/valid_text_seq.csv')
df_train.head()



,input_str,label
0,0000154364642718159661428002624223132284159626...,0
1,0004641596369515436422262614110471596262476161...,0
2,0001543626215965999614422464135806142624051159...,0
3,0000154364224641238614262159689561596284351061...,1
4,0004641899422154362069015966142624761262159661...,1


# 100%

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the datasets
df_train = pd.read_csv('/content/train_text_seq.csv')
df_valid = pd.read_csv('/content/valid_text_seq.csv')

# Extract input sequences and labels
X_train = df_train['input_str'].values
y_train = df_train['label'].values
X_valid = df_valid['input_str'].values
y_valid = df_valid['label'].values

# Encode the digits using LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(list('0123456789'))

# Convert input_str into sequences of encoded digits, removing the first three zeroes
def encode_sequence(sequence):
    return label_encoder.transform(list(sequence[3:]))  # Remove first 3 zeros

X_train_encoded = [encode_sequence(seq) for seq in X_train]
X_valid_encoded = [encode_sequence(seq) for seq in X_valid]

# Pad the sequences to ensure they all have the same length (47 in this case)
X_train_padded = pad_sequences(X_train_encoded, maxlen=47, padding='post')
X_valid_padded = pad_sequences(X_valid_encoded, maxlen=47, padding='post')

# Convert the labels to categorical (0 or 1)
y_train_categorical = to_categorical(y_train, num_classes=2)
y_valid_categorical = to_categorical(y_valid, num_classes=2)

# Build the LSTM model with more parameters while staying under 10,000
model = Sequential()
model.add(Embedding(input_dim=10, output_dim=16, input_length=47))  # 10 unique digits, embedding size 16
model.add(LSTM(32, return_sequences=True))  # Bidirectional LSTM with 32 units, return sequences
model.add(Dropout(0.3))  # Dropout layer to prevent overfitting
model.add(LSTM(16))  # Additional LSTM layer with 16 units
model.add(Dense(16, activation='relu'))  # Dense layer with 16 units
model.add(Dense(2, activation='softmax'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_categorical,
          validation_data=(X_valid_padded, y_valid_categorical),
          epochs=200, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_valid_padded, y_valid_categorical)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/200
222/222 ━━━━━━━━━━━━━━━━━━━━ 14s 42ms/step - accuracy: 0.5052 - loss: 0.6930 - val_accuracy: 0.6074 - val_loss: 0.6728
Epoch 2/200
222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - accuracy: 0.6109 - loss: 0.6599 - val_accuracy: 0.6278 - val_loss: 0.6319
Epoch 3/200
222/222 ━━━━━━━━━━━━━━━━━━━━ 12s 39ms/step - accuracy: 0.6336 - loss: 0.6392 - val_accuracy: 0.6360 - val_loss: 0.6250
Epoch 4/200
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.6402 - loss: 0.6317 - val_accuracy: 0.6524 - val_loss: 0.6302
Epoch 5/200
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - accuracy: 0.6378 - loss: 0.6304 - val_accuracy: 0.6667 - val_loss: 0.6143
Epoch 6/200
222/222 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.6323 - loss: 0.6340 - val_accuracy: 0.6708 - val_loss: 0.6062
Epoch 7/200
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 42ms/step - accuracy: 0.6495 - loss: 0.6239 - val_accuracy: 0.6728 - val_loss: 0.6133
Epoch 8/200
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - accuracy: 0.6487 - loss: 0.6214

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 47, 16)              │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 47, 32)              │           6,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 47, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │              34 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,624 (115.72 KB)

 Trainable params: 9,874 (38.57 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 19,750 (77.15 KB)

# 80%

# 60%

# 40%

# 20%